#### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
    
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [45]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import time
import re


In [46]:
def hh_pars_page(hh_xml, data):
    ''' 
    Парсинг HH-lxml страницы и добавление данных в pd.data
    '''

    list_block = hh_xml.find_all('div',{"class":"vacancy-serp-item"})


    for div in list_block:
        vacancy = div.find('a', {'class': 'bloko-link HH-LinkModifier'}).text
        #print(vacancy)
        vacancy_link = div.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
        #print(vacancy_link)
        employer = div.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).text
        #print(employer)

        compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})

        if not compensation:
            compensation1 = np.NaN
            compensation2 = np.NaN
        else:
            compensation = compensation.text.replace('.', '').replace(' ','').replace('руб','')
           # print(compensation)
            if '-' in compensation:
                compensations = compensation.split('-')

                compensation1 = compensations[0] #.replace(r'\D', '').replace(r'\xa', '')
                compensation1 = re.sub(r'\D', '',compensation1)
                compensation2 = compensations[1] #.replace(r'\D', '').replace(r'\xa', '')
                compensation2 = re.sub(r'\D', '',compensation2)
                
                compensation1 = int(compensation1)
                compensation1 = int(compensation2)


            elif 'от' in compensation:
                compensation1 = compensation.replace('от','').replace(r'\xa', '')
                compensation1 = re.sub(r'\D', '',compensation1)
                compensation1 = int(compensation1)
                compensation2 = np.NaN
            elif 'до' in compensation:
                compensation1 = np.NaN
                compensation2 = compensation.replace('до','').replace(' ', '')
                compensation2 = re.sub(r'\D', '',compensation2)
                compensation1 = int(compensation2)

        #print(compensation1, compensation2)


        data = data.append({'vacancy':[vacancy], 'min_salary':[compensation1], 'max_salary':[compensation2], 'vacancy_link':[vacancy_link], 'link':['https://hh.ru'], 
                            'employer':[employer]}, ignore_index=True)

        #compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    return(data)
    

In [47]:
def sj_pars_page(sj_xml, data):
    ''' 
    Парсинг SJ-lxml страницы и добавление данных в pd.data
    '''

    list_block = sj_xml.find_all('div',{"class":"_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"})
    pprint(list_block)
    for div in list_block:
        vacancy = div.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).text
        #print(vacancy)
        vacancy_link = div.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).findParent()['href']
        vacancy_link = 'https://www.superjob.ru' + vacancy_link                               
        #print(vacancy_link)
        employer = div.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI'}).text
        #print(employer)     

        compensation = div.find('span', {'class': '_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).text
        #print(compensation)
        compensation1 = np.NaN
        compensation2 = np.NaN
        if '—' in compensation:

            compensation1 = compensation.split('—')[0]
            compensation2 = compensation.split('—')[1]
            compensation1 = re.sub(r'\D', '',compensation1)
            compensation2 = re.sub(r'\D', '',compensation2)
            compensation1 = int(compensation1)
            compensation1 = int(compensation2)

        elif 'от' in compensation:
            compensation1 = compensation[3:]
            compensation1 = re.sub(r'\D', '',compensation1)
            #print("compensation1=", compensation1)
            compensation1 = int(compensation1)
            compensation2 = np.NaN
        elif 'договорённости' in compensation:
            compensation1 = np.NaN
            compensation2 = np.NaN
        elif 'до' in compensation:
            compensation1 = np.NaN
            compensation2 = compensation[3:]
            compensation1 = int(compensation2)
        
        
        data = data.append({'vacancy':[vacancy], 'min_salary':[compensation1], 'max_salary':[compensation2], 'vacancy_link':[vacancy_link], 'link':['https://www.superjob.ru'], 
                            'employer':[employer]}, ignore_index=True)

        #compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    return(data)
    

In [48]:
# Общие параметры поиска: текст вакансии и глубина
search_text = "методолог"
pages = 4
data = pd.DataFrame()

In [49]:
#  Обработка HH

main_link = ('https://hh.ru')
method = "/search/vacancy?area=1&st=searchVacancy&text="
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

url = main_link+method+search_text
print(main_link+method+search_text)

session = requests.Session()

for i in range(pages):
    html = session.get(url, headers=headers).text
    
    hh_xml = bs(html,'lxml')
    data = hh_pars_page(hh_xml, data)
    url = main_link + hh_xml.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href']
    
    time.sleep(1)
    
    

https://hh.ru/search/vacancy?area=1&st=searchVacancy&text=методолог


In [50]:
#  Обработка SJ

main_link = ('https://www.superjob.ru')
method = "/vacancy/search/?keywords="
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

url = main_link+method+search_text
print(main_link+method+search_text)

session = requests.Session()


for i in range(pages):
    html = session.get(url, headers=headers).text
    
    sj_xml = bs(html,'lxml')
    data = sj_pars_page(sj_xml, data)
    url = main_link + sj_xml.find('a', {'class': 'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-dalshe'})['href']
    
    time.sleep(1)


https://www.superjob.ru/vacancy/search/?keywords=методолог
[<div class="_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"><div class="_36e54 _2ntFx"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"></div><div class="_2g1F-"><div><div class="_1Tocb"><a class="icMQ_ f-test-link-" href="/clients/piccasushivok-2535003.html" target="_self"><div class="lazyload-placeholder" style="height:44px"></div></a></div><div class="_3syPg _1_bQo _2FJA4"><div class="_2g1F-"><a class="icMQ_ _1QIBo f-test-link-Specialist_po_razrabotke_kursov_Razrabotchik_distancionnyh_kursov_Metodolog _2JivQ _3dPok" href="/vakansii/specialist-po-razrabotke-kursov-32758228.html" target="_blank"><div class="_3mfro CuJz5 PlM3e _2JVkc _3LJqf"><span class="_1rS-s">Специалист</span> по разработке курсов / Разработчик дистанционных курсов / <span class="_1rS-s">Методолог</span></div></a></div><div class="_2g1F-"><div class="_2XXYS _2cxK3"><div class="_2g1F-"><span cl

[<div class="_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"><div class="_36e54 _2ntFx"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"></div><div class="_2g1F-"><div><div class="_1Tocb"></div><div class="_3syPg _1_bQo _2FJA4"><div class="_2g1F-"><a class="icMQ_ _1QIBo f-test-link-Zamestitel_direktora_po_metodicheskoj_rabote_metodist_v_kolledzhe _2JivQ _3dPok" href="/vakansii/zamestitel-direktora-po-metodicheskoj-rabote-31102536.html" target="_blank"><div class="_3mfro CuJz5 PlM3e _2JVkc _3LJqf">Заместитель директора по методической работе, <span class="_1rS-s">методист</span> в колледже</div></a></div><div class="_2g1F-"><div class="_2XXYS _2cxK3"><div class="_2g1F-"><span class="_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI"><a class="icMQ_ _205Zx f-test-link-Kolledzh_MIREA Vm5jz" href="/clients/kolledzh-mirea-2637274.html" target="_self">Колледж МИРЭА</a></span></div><div class="_2g1F

[<div class="_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"><div class="_36e54 _2ntFx"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"></div><div class="_2g1F-"><div><div class="_1Tocb"></div><div class="_3syPg _1_bQo _2FJA4"><div class="_2g1F-"><a class="icMQ_ _1QIBo f-test-link-Metodist_VKK _2JivQ _3dPok" href="/vakansii/metodist-vkk-29229725.html" target="_blank"><div class="_3mfro CuJz5 PlM3e _2JVkc _3LJqf"><span class="_1rS-s">Методист</span> ВКК</div></a></div><div class="_2g1F-"><div class="_2XXYS _2cxK3"><div class="_2g1F-"><span class="_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI"><a class="icMQ_ _205Zx f-test-link-Kolledzh_informatiki_i_programmirovaniya Vm5jz" href="/clients/kolledzh-informatiki-i-programmirovaniya-1992275.html" target="_self">Колледж информатики и программирования</a></span></div><div class="_2g1F-"><span class="_3mfro f-test-text-company-item-location _9fX

[<div class="_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"><div class="_36e54 _2ntFx"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_1_bQo _2FJA4 _1iZ5S"><div class="_2g1F-"><span class="_1kf9U _1VKE- f-test-badge _2FkY7">Отклик без резюме</span></div></div></div><div class="_2g1F-"><div><div class="_1Tocb"><a class="icMQ_ f-test-link-" href="/clients/sberbank-rossii-297921.html" target="_self"><div class="lazyload-placeholder" style="height:44px"></div></a></div><div class="_3syPg _1_bQo _2FJA4"><div class="_2g1F-"><a class="icMQ_ _1QIBo f-test-link-Analitik_DWH _2JivQ _3dPok" href="/vakansii/analitik-dwh-32594483.html" target="_blank"><div class="_3mfro CuJz5 PlM3e _2JVkc _3LJqf"><span class="_1rS-s">Аналитик</span> DWH</div></a></div><div class="_2g1F-"><div class="_2XXYS _2cxK3"><div class="_2g1F-"><span class="_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI"><a class="

In [51]:
data.head()

,employer,link,max_salary,min_salary,vacancy,vacancy_link
0,[ Всероссийский Банк Развития Регионов (ВБРР)],[https://hh.ru],[nan],[nan],[Методолог кадрового администрирования],[https://hh.ru/vacancy/34240183?query=%D0%BC%D...
1,"[ Ренессанс cтрахование, Группа]",[https://hh.ru],[nan],[nan],[Методолог/Руководитель направления методологи...,[https://hh.ru/vacancy/34005950?query=%D0%BC%D...
2,"[ Ренессанс cтрахование, Группа]",[https://hh.ru],[nan],[nan],[Ведущий специалист/Риск-менеджер в Управление...,[https://hh.ru/vacancy/34195483?query=%D0%BC%D...
3,[ FinFort],[https://hh.ru],[nan],[70000],[Методолог],[https://hh.ru/vacancy/33436906?query=%D0%BC%D...
4,[ Ростелеком - Центры обработки данных],[https://hh.ru],[nan],[nan],[Методолог],[https://hh.ru/vacancy/32082936?query=%D0%BC%D...


In [52]:
data.to_pickle('vacancy_pd.pkl')

In [53]:
data_import = pd.read_pickle('vacancy_pd.pkl')
data_import

,employer,link,max_salary,min_salary,vacancy,vacancy_link
0,[ Всероссийский Банк Развития Регионов (ВБРР)],[https://hh.ru],[nan],[nan],[Методолог кадрового администрирования],[https://hh.ru/vacancy/34240183?query=%D0%BC%D...
1,"[ Ренессанс cтрахование, Группа]",[https://hh.ru],[nan],[nan],[Методолог/Руководитель направления методологи...,[https://hh.ru/vacancy/34005950?query=%D0%BC%D...
2,"[ Ренессанс cтрахование, Группа]",[https://hh.ru],[nan],[nan],[Ведущий специалист/Риск-менеджер в Управление...,[https://hh.ru/vacancy/34195483?query=%D0%BC%D...
3,[ FinFort],[https://hh.ru],[nan],[70000],[Методолог],[https://hh.ru/vacancy/33436906?query=%D0%BC%D...
4,[ Ростелеком - Центры обработки данных],[https://hh.ru],[nan],[nan],[Методолог],[https://hh.ru/vacancy/32082936?query=%D0%BC%D...
5,[ООО СТРАТЕГИЯ],[https://hh.ru],[150000],[150000],[Методолог/Финансовый Бизнес аналитик],[https://hh.ru/vacancy/33223198?query=%D0%BC%D...
6,"[ПАО «Группа Черкизово», Центральный офис]",[https://hh.ru],[100000],[100000],[Методолог-консультант],[https://hh.ru/vacancy/34276196?query=%D0%BC%D...
7,[АО БАНК ИНТЕЗА],[https://hh.ru],[nan],[nan],[Главный методолог],[https://hh.ru/vacancy/33875879?query=%D0%BC%D...
8,[ООО Озерная Горнорудная Компания],[https://hh.ru],[nan],[nan],[Главный специалист по методологии],[https://hh.ru/vacancy/33932248?query=%D0%BC%D...
9,[ DM Basis],[https://hh.ru],[60000],[60000],[Специалист по документообороту / Методолог],[https://hh.ru/vacancy/34059280?query=%D0%BC%D...
